# LFT 04 — Qudit Bridge: Simplex $\Delta^{N-1}$ ↔ Sum-zero Space $V$
This notebook proves and verifies the **qudit/simplex bridge** used in LFT:

1. The affine map $\phi: \Delta^{N-1}\to V$, $\phi(p)=p-\tfrac{1}{N}\mathbf{1}$, takes the probability simplex to the **sum-zero space** $V=\{x\in\mathbb{R}^N:\sum x_i=0\}\cong\mathbb{R}^{N-1}$.
2. For **generic** $p$ (all coordinates distinct), the $S_N$-orbit $\{\sigma\cdot p\}$ maps to the **vertex set** of a **permutohedron** in $V$; its edges correspond to **adjacent transpositions** (Coxeter generators of $A_{N-1}$).
3. We generate and plot the orbits for **N=4** and **N=5**, save a CSV for reproducibility (N=4), and verify counts and Cayley adjacency.


## 1. Affine isomorphism $\phi: \Delta^{N-1} \to V$
**Proposition.** The map $\phi(p)=p-\tfrac{1}{N}\mathbf{1}$ is an affine isomorphism between the simplex hyperplane $\{p:\sum p_i=1\}$ and the sum-zero subspace $V$. It commutes with the action of $S_N$ (coordinate permutations).

**Proof.** $\sum_i \phi(p)_i = \sum_i p_i - 1 = 0$, so $\phi(\Delta^{N-1})\subset V$. The map is a translation followed by identity on directions within the hyperplane, hence bijective onto $V$. For any permutation matrix $P$ (representing $\sigma\in S_N$), $\phi(Pp) = Pp - \tfrac{1}{N}\mathbf{1} = P(p-\tfrac{1}{N}\mathbf{1}) = P\phi(p)$, so it commutes with $S_N$. ∎

In [ ]:
import numpy as np
import itertools, networkx as nx

def phi_to_V(p):
    p = np.asarray(p, dtype=float)
    return p - p.mean()  # since sum p_i = 1, mean = 1/N

def orbit_SN_points(p):
    N = len(p)
    perms = list(itertools.permutations(range(N)))
    points = np.array([p[list(s)] for s in perms], dtype=float)
    return points, perms

def cayley_adjacent_graph_from_perms(perms):
    N = len(perms[0])
    idx = {p:i for i,p in enumerate(perms)}
    G = nx.Graph()
    G.add_nodes_from(range(len(perms)))
    gens = [(i, i+1) for i in range(N-1)]
    for p in perms:
        u = idx[p]
        for (i,j) in gens:
            q = list(p)
            q[i], q[j] = q[j], q[i]
            v = idx[tuple(q)]
            if u < v:
                G.add_edge(u, v)
    return G

def pca_projection(X, k):
    Xc = X - X.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Xc, full_matrices=False)
    return Xc @ Vt[:k].T


## 2. N=4: Orbit plot, CSV, and adjacency checks

We choose a **generic** probability vector (all entries distinct), compute its $S_4$ orbit, map to $V$, and visualize in 3D (no projection needed because $V\cong\mathbb{R}^3$). We also save the orbit as CSV for reproducibility.

In [ ]:
import os, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

os.makedirs('./outputs', exist_ok=True)

N = 4
p4 = np.array([0.07, 0.19, 0.29, 0.45])
p4 = p4 / p4.sum()  # ensure sums to 1
pts4, perms4 = orbit_SN_points(p4)
V4 = np.array([phi_to_V(x) for x in pts4])  # already in R^4 but sum-zero hyperplane

# Build adjacent Cayley graph
G4 = cayley_adjacent_graph_from_perms(perms4)
nodes4, edges4 = V4.shape[0], G4.number_of_edges()
print({'N':4,'nodes':nodes4,'edges_adjacent':edges4,'avg_degree':2*edges4/nodes4})
assert nodes4 == 24 and edges4 == 36

# Reduce to V ≅ R^3 by an orthonormal basis for sum-zero space
def sum_zero_basis(N):
    diffs = np.zeros((N, N-1))
    for i in range(N-1):
        diffs[i, i] = 1.0
        diffs[i+1, i] = -1.0
    U,S,Vt = np.linalg.svd(diffs, full_matrices=False)
    return U  # N x (N-1)

B4 = sum_zero_basis(4)
V4_coords = V4 @ B4  # (24,3)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(V4_coords[:,0], V4_coords[:,1], V4_coords[:,2], s=30)
for u, v in G4.edges():
    x = [V4_coords[u,0], V4_coords[v,0]]
    y = [V4_coords[u,1], V4_coords[v,1]]
    z = [V4_coords[u,2], V4_coords[v,2]]
    ax.plot(x, y, z, linewidth=0.7)
ax.set_title('S4 orbit on Δ^3 mapped to V (Π3 vertices)')
plt.tight_layout()
plt.savefig('./outputs/N4_simplex_orbit_permutohedron.png', dpi=150)
plt.close()
print('Saved ./outputs/N4_simplex_orbit_permutohedron.png')

# Save CSV (per-vertex probabilities and V-coordinates)
import pandas as pd
df4 = pd.DataFrame({
    'p0': pts4[:,0], 'p1': pts4[:,1], 'p2': pts4[:,2], 'p3': pts4[:,3],
    'Vx': V4_coords[:,0], 'Vy': V4_coords[:,1], 'Vz': V4_coords[:,2]
})
df4.to_csv('./outputs/N4_orbit_simplex_permutohedron.csv', index=False)
print('Saved ./outputs/N4_orbit_simplex_permutohedron.csv')

## 3. N=5: Orbit plot (PCA to 3D) and adjacency checks

For **N=5**, $V\cong\mathbb{R}^4$. We visualize the 120-point orbit using a 3D PCA projection and overlay adjacent edges (240).

In [ ]:
N = 5
p5 = np.array([0.06, 0.13, 0.21, 0.27, 0.33])
p5 = p5 / p5.sum()
pts5, perms5 = orbit_SN_points(p5)
V5 = np.array([phi_to_V(x) for x in pts5])

G5 = cayley_adjacent_graph_from_perms(perms5)
nodes5, edges5 = V5.shape[0], G5.number_of_edges()
print({'N':5,'nodes':nodes5,'edges_adjacent':edges5,'avg_degree':2*edges5/nodes5})
assert nodes5 == 120 and edges5 == 240

# Project V (sum-zero hyperplane in R^5) → R^3 for visualization
def pca_projection(X, k):
    Xc = X - X.mean(axis=0, keepdims=True)
    U,S,Vt = np.linalg.svd(Xc, full_matrices=False)
    return Xc @ Vt[:k].T

P3 = pca_projection(V5, 3)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(P3[:,0], P3[:,1], P3[:,2], s=15)
for u, v in G5.edges():
    x = [P3[u,0], P3[v,0]]
    y = [P3[u,1], P3[v,1]]
    z = [P3[u,2], P3[v,2]]
    ax.plot(x, y, z, linewidth=0.5)
ax.set_title('S5 orbit on Δ^4 mapped to V (Π4 vertices), PCA to 3D')
plt.tight_layout()
plt.savefig('./outputs/N5_simplex_orbit_permutohedron_PCA3.png', dpi=150)
plt.close()
print('Saved ./outputs/N5_simplex_orbit_permutohedron_PCA3.png')

## 4. Conclusions
- The simplex hyperplane and sum-zero space are **affinely isomorphic** via $\phi$.
- Generic simplex orbits under $S_N$ produce the **permutohedron vertex sets** in $V$.
- **Edges = adjacent swaps** (Coxeter generators), verified by counts (N=4: 24/36, N=5: 120/240).
- This establishes the **qudit-population** reading of the $N-1$ axes as **population differences** in $V$ (the “diagonal sector”).